In [1]:
import re
import sys
import datetime
sys.path.append(r"D:\往期\QHJ\ZERO")
sys.path.append(r"E:\dataparse\Python_DATA_PARSE\QHJ\ZERO")
from pyquery import PyQuery as pq
import requests
from urllib.parse import urlencode,parse_qs
import numpy as np
import pandas as pd
import redis

import json
from tools import *



# 测试
# 测试
# conn = redis.Redis(**REDIS_TEST_DIC)


conn = redis.Redis(**REDIS_MALL_ORDER_DIC)

In [17]:
l = []
# data = pd.read_excel(r"C:\Users\qhj01\Desktop\趣领运单号12-15.xlsx",converters = {u'lqzydm':str,u'lqyxsm':str})
# l.append(data)
data = pd.read_excel(r"C:\Users\qhj01\Desktop\趣领运单号12-16.xlsx",converters = {'订单号':str,})
l.append(data)
data = pd.concat(l)

In [18]:
len(data)

12

In [4]:
headers = {
    "accept": "*/*",
"accept-encoding": "gzip, deflate, br",
"accept-language": "zh-CN,zh;q=0.9,en;q=0.8",
"cache-control": "no-cache",
# "content-length": "53",
"content-type": "application/x-www-form-urlencoded; charset=UTF-8",
"cookie": "PHPSESSID=m442n8rq407kg529gn37tf4vkd",
"origin": "https://app0001.yrapps.cn",
"pragma": "no-cache",
"referer": "https://app0001.yrapps.cn/admin/order/order_delivery/order_id/7801733718286446.html",
"sec-fetch-mode": "cors",
"sec-fetch-site": "same-origin",
"user-agent": "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36",
"x-requested-with": "XMLHttpRequest",
}

url = r"https://app0001.yrapps.cn/admin/order/send_order.html"



In [12]:
data.columns

Index(['订单号', '商品ID', '商品名', '数量', '规格', '单位', '收件人', '收货地址', '联系方式', '运单号',
       '快递方式'],
      dtype='object')

In [19]:
data

,订单号,商品ID,商品名,数量,规格,单位,收件人,收货地址,联系方式,运单号,快递方式
0,0070989020623815,S0003P0001C0003P01E01,三金护肤面膜 维稳修护 强化屏障(护肤),1,5片*25g,盒,汤心如,湖南省株洲市芦淞区五里墩，株洲南方航空高级技工学校北校区,15697336645,75321121599569,中通
1,3562409723215555,S0003P0001C0001P01E01,三金补水面膜 镇静维稳 深层补水(补水),1,5片*25g,盒,汤意,湖南省长沙市开福区湖南省长沙市开福区福元路河马公园长沙研美整形医疗,15116324813,75321121599571,中通
2,0405382866061700,S0003P0001C0003P01E01,三金护肤面膜 维稳修护 强化屏障(护肤),1,5片*25g,盒,李水,湖南省长沙市开福区万达广场C1栋4712室,15387533103,75321121599648,中通
3,5036756284013369,S0003P0001C0004P01E01,三金水光针(水光针),1,10ml/支,盒,刘以茹,省市区湖南省长沙县星沙大道39号财富港湾一楼华夏银行,13973134177,3102993740272,韵达
4,6394316946544369,S0003P0001C0001P01E01,三金补水面膜 镇静维稳 深层补水(补水),1,5片*25g,盒,刘芳汝,湖南省常德市桃源县丹阳和庭2期1202室,18774355843,75321121599584,中通
5,2178604481975617,S0003P0001C0001P01E01,三金补水面膜 镇静维稳 深层补水(补水),1,5片*25g,盒,龙芳,湖南省长沙市雨花区长岭宾馆(长沙市雨花区)水电八局红楼菜鸟驿站,15116395127,75321121599597,中通
6,2514409760504324,S0003P0001C0001P01E01,三金补水面膜 镇静维稳 深层补水(补水),1,5片*25g,盒,孙珺,湖南省长沙市雨花区韶山南路路桥集团小区南院38栋104,18711089020,75321121599607,中通
7,6251410337128243,S0003P0001C0001P01E01,三金补水面膜 镇静维稳 深层补水(补水),1,5片*25g,盒,谢志华,湖南省长沙市芙蓉区德政街168号教师二村11栋206,15580907335,75321121599610,中通
8,6725829012631395,S0003P0001C0002P01E01,三金美肤面膜 美白功效 健康肤色(美肤),1,5片*25g,盒,张惠,湖南省长沙市芙蓉区德政街168号教室二村11栋三单元,15675887750,75321121599622,中通
9,2221048000449924,S0003P0001C0001P01E01,三金补水面膜 镇静维稳 深层补水(补水),1,5片*25g,盒,周瑶,湖南省长沙市望城区湖南省长沙市望城区金桥国际未来城4栋7楼,17526399658,75321121599635,中通


In [20]:
for i,row in data.iterrows():

    data = {
        "express_info": row['快递方式'],
        "express_id": row['运单号'],
        "order_id": row['订单号']
    }
    print(data)
    requests.post(url,headers=headers,data=data)
    
#     requests.post()
    
    
    

{'express_info': '中通', 'express_id': 75321121599569, 'order_id': '0070989020623815'}
{'express_info': '中通', 'express_id': 75321121599571, 'order_id': '3562409723215555'}
{'express_info': '中通', 'express_id': 75321121599648, 'order_id': '0405382866061700'}
{'express_info': '韵达', 'express_id': 3102993740272, 'order_id': '5036756284013369'}
{'express_info': '中通', 'express_id': 75321121599584, 'order_id': '6394316946544369'}
{'express_info': '中通', 'express_id': 75321121599597, 'order_id': '2178604481975617'}
{'express_info': '中通', 'express_id': 75321121599607, 'order_id': '2514409760504324'}
{'express_info': '中通', 'express_id': 75321121599610, 'order_id': '6251410337128243'}
{'express_info': '中通', 'express_id': 75321121599622, 'order_id': '6725829012631395'}
{'express_info': '中通', 'express_id': 75321121599635, 'order_id': '2221048000449924'}
{'express_info': '中通', 'express_id': 75321121599648, 'order_id': '9535818429943523'}
{'express_info': '中通', 'express_id': 75321121599656, 'order_id': '

In [9]:

for i,row in data.iterrows():
    r= conn.sismember('order:new',row['订单号'])
    if not r:
        print(row['订单号'])
    else:
        print(r)

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


In [ ]:
stop


三金_新订单 2019-12-12 15_37_45.xlsx
三金_新订单 2019-12-13 15_38_09.xlsx
三金_新订单 2019-12-14 15_46_34.xlsx
三金_新订单 2019-12-15 15_36_53.xlsx


In [ ]:
for i,row in data.iterrows():
    spp_id = row['商品ID'][:5]
    print(row['单号'])
    r = conn.smove("order:old:%s" %(spp_id),
               "ORDER:RC:%s" %(spp_id)
               ,row['订单号'])
    print(r)

In [18]:
data = pd.read_excel(r"E:\qhj\三金外发\三金_新订单 2019-12-12 15_37_45.xlsx",index=False)
conn.sadd("order:old",*data['订单号'].tolist())
data = pd.read_excel(r"E:\qhj\三金外发\三金_新订单 2019-12-13 15_38_09.xlsx",index=False)
conn.sadd("order:old",*data['订单号'].tolist())


20

In [12]:
data['key']

0     7801733718286446_S0003
1       19121251101531_S0003
2       19121248494810_S0003
3       19121253985148_S0003
4       19121249485597_S0003
5       19121252575210_S0003
6       19121257485710_S0003
7       19121298101101_S0003
8       19121210253569_S0003
9       19121297579856_S0003
10      19121298574910_S0003
11      19121297101535_S0003
12      19121249971025_S0003
13      19121250575757_S0003
14      19121248101504_S0003
15      19121255999954_S0003
16      19121210210051_S0003
17      19121251101505_S0003
18      19121210052989_S0003
19      19121250985210_S0003
20      19121299971024_S0003
21      19121210210010_S0003
0     5609222359417374_S0003
1     9784424719606776_S0003
2     3400138912332780_S0003
3     7620266221650637_S0003
4     8171817913572174_S0003
5     2758468066946996_S0003
6      501250902978832_S0003
7     2163778663397136_S0003
               ...          
10    9076677957182044_S0003
11    1015787224552650_S0003
12     149891054111246_S0003
13    26849461

In [19]:

f_dir = r"D:\奇货居\work\外发订单\新订单\已发"
l = []
for file in os.listdir(f_dir):
    if file.startswith("三金"):
        print(file)
        file_path = os.path.join(f_dir,file)
        l.append(pd.read_excel(file_path))

data = pd.concat(l)

# data = pd.read_excel(r"D:\奇货居\work\外发订单\新订单\已发\三金_新订单 2019-12-15 15_36_53.xlsx")
# data['key'] = data['订单号'].astype(str) + "_S0003"

conn.sadd("order:old",*data['订单号'].tolist())



三金_新订单 2019-12-12 15_37_45.xlsx
三金_新订单 2019-12-13 15_38_09.xlsx
三金_新订单 2019-12-14 15_46_34.xlsx
三金_新订单 2019-12-15 15_36_53.xlsx


62